# Entire Serial Implementation with Wordlist

In [1]:
import sys
import time

def keyword_picker():
    print("Enter the Key")
    keyword = input("Enter key: ").upper()
    for i in range(len(keyword)):
        pos = i
        for j in range(len(keyword)):
            if pos == j:
                continue
            elif keyword[i] == keyword[j]:
                print("Letter {} repeating in keyword".format(keyword[i]))
                sys.exit()
    temp = ""
    for i in range(len(keyword)):
        temp += keyword[i]
    for i in range(26):
        temp += chr(i+65)

    alpha_with_key = ""
    for i in range(len(temp)):
        found = False
        for j in range(len(alpha_with_key)):
            if temp[i] == alpha_with_key[j]:
                found = True
                break
        if not found:
            alpha_with_key += temp[i]
    return alpha_with_key


def encryption(alpha_key, alpha,word):
    message = word.upper()

    encrypted_text = ""
    for i in range(len(message)):
        if message[i] == chr(32):
            encrypted_text += " "
        else:
            counter = 0
            for j in range(len(alpha)):
                if message[i] == alpha[j]:
                    encrypted_text += alpha_key[counter]
                    break
                else:
                    counter += 1
    #print("Encrypted Message: {}".format(encrypted_text))


def decryption(alpha_key, alpha,word):
    
    message = word.upper()

    decrypted_text = ""
    for i in range(len(message)):
        if message[i] == chr(32):
            decrypted_text += " "
        else:
            counter = 0
            for j in range(len(alpha)):
                if message[i] == alpha_key[j]:
                    decrypted_text += alpha[counter]
                    break
                else:
                    counter += 1
    #print("Decrypted Text: {}".format(decrypted_text))


def main():     
                
    alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alpha_key = keyword_picker()

    choice = int(input("1. Encryption\n2. Decryption\nChoose(1,2): "))
    
    if choice == 1:
        start_time = time.time()
        print("---Encrypting---")
        with open('dataset.txt','r') as file:
            enc=[encryption(alpha_key,alpha,word) for word in file.read().split()]
        end_time=time.time()
        print("Total number of words= ",len(enc))
        print("Execution time= ",end_time-start_time)

    elif choice == 2:
        start_time = time.time()
        print("---Decrypting---")
        with open('dataset.txt','r') as file:
            dec=[encryption(alpha_key,alpha,word) for word in file.read().split()]
        end_time=time.time()
        print("Total number of words= ",len(dec))
        print("Execution time= ",end_time-start_time)
    else:
        print("Incorrect Choice")

if __name__ == "__main__":
    main()

Enter the Key
Enter key: keywords
1. Encryption
2. Decryption
Choose(1,2): 1
---Encrypting---
Total number of words=  12035182
Execution time=  163.87890672683716


# Joblib

In [29]:
import sys
import time
import joblib
import dask.dataframe as dd
from joblib import Parallel, delayed

def keyword_picker():
    print("Enter the Key")
    keyword = input("Enter key: ").upper()
    for i in range(len(keyword)):
        pos = i
        for j in range(len(keyword)):
            if pos == j:
                continue
            elif keyword[i] == keyword[j]:
                print("Letter {} repeating in keyword".format(keyword[i]))
                sys.exit()
    temp = ""
    for i in range(len(keyword)):
        temp += keyword[i]
    for i in range(26):
        temp += chr(i+65)

    alpha_with_key = ""
    for i in range(len(temp)):
        found = False
        for j in range(len(alpha_with_key)):
            if temp[i] == alpha_with_key[j]:
                found = True
                break
        if not found:
            alpha_with_key += temp[i]
    return alpha_with_key


def encryption(alpha_key, alpha,word):
    message = word.upper()

    encrypted_text = ""
    for i in range(len(message)):
        if message[i] == chr(32):
            encrypted_text += " "
        else:
            counter = 0
            for j in range(len(alpha)):
                if message[i] == alpha[j]:
                    encrypted_text += alpha_key[counter]
                    break
                else:
                    counter += 1
    return encrypted_text


def decryption(alpha_key, alpha,word):
    
    message = word.upper()

    decrypted_text = ""
    for i in range(len(message)):
        if message[i] == chr(32):
            decrypted_text += " "
        else:
            counter = 0
            for j in range(len(alpha)):
                if message[i] == alpha_key[j]:
                    decrypted_text += alpha[counter]
                    break
                else:
                    counter += 1
    return decrypted_text


def main():     
                
    alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alpha_key = keyword_picker()

    choice = int(input("1. Encryption\n2. Decryption\nChoose(1,2): "))
    
    if choice == 1:
        words=dd.read_csv('dataset.txt').repartition(npartitions=8)
        start_time = time.time()
        print("---Encrypting---")
        with open('dataset.txt','r') as file:
            Parallel(8)(delayed(encryption)(alpha_key,alpha,word) for word in file.read().split())
        end_time=time.time()
        print("Execution time= ",end_time-start_time)

    elif choice == 2:
        words=dd.read_csv('dataset.txt').repartition(npartitions=8)
        start_time = time.time()
        print("---Decrypting---")
        words=words.apply(lambda x: decryption(alpha_key,alpha,str(x)),meta=('str'),axis=1).compute()
        end_time=time.time()
        print("Execution time= ",end_time-start_time)
    else:
        print("Incorrect Choice")

if __name__ == "__main__":
    main()

Enter the Key
Enter key: keywords
1. Encryption
2. Decryption
Choose(1,2): 1
---Encrypting---
Execution time=  99.19095253944397


# Dask Dataframe Implementation

#### Dask Client Setup

In [22]:
import dask.dataframe as dd
import dask
from dask.distributed import Client, progress
client = Client(threads_per_worker=8, n_workers=1)
client

Client Scheduler: tcp://127.0.0.1:3117 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 8 Memory: 17.04 GB


In [59]:
import sys
import time
import dask.array as da
from dask.bag import read_text

def keyword_picker():
    print("Enter the Key")
    keyword = input("Enter key: ").upper()
    for i in range(len(keyword)):
        pos = i
        for j in range(len(keyword)):
            if pos == j:
                continue
            elif keyword[i] == keyword[j]:
                print("Letter {} repeating in keyword".format(keyword[i]))
                sys.exit()
    temp = ""
    for i in range(len(keyword)):
        temp += keyword[i]
    for i in range(26):
        temp += chr(i+65)

    alpha_with_key = ""
    for i in range(len(temp)):
        found = False
        for j in range(len(alpha_with_key)):
            if temp[i] == alpha_with_key[j]:
                found = True
                break
        if not found:
            alpha_with_key += temp[i]
    return alpha_with_key


def encryptionpd(word,alpha_key, alpha):
    message = word[0].str.upper()
    print(message)
    for j in range(len(message)):
        encrypted_text = ""
        for i in range(len(message.loc[j])):
            if message.loc[j][i] == chr(32):
                encrypted_text += " "
            else:
                counter = 0
                for k in range(len(alpha)):
                    if message.loc[j][i] == alpha[k]:
                        encrypted_text += alpha_key[counter]
                        break
                    else:
                        counter += 1
        message.loc[j]=encrypted_text
    return message

def decryptionpd(word,alpha_key, alpha):
    
    message = word[0].compute().str.upper()

    decrypted_text = ""
    for j in range(len(message)):
        encrypted_text = ""
        for i in range(len(message)):
            if message.loc[j][i] == chr(32):
                decrypted_text += " "
            else:
                counter = 0
                for k in range(len(alpha)):
                    if message.loc[j][i] == alpha_key[k]:
                        decrypted_text += alpha[counter]
                        break
                    else:
                        counter += 1
        message.loc[j]=decrypted_text
    return message

def encryption(word,alpha_key, alpha):
    message = word.upper()

    encrypted_text = ""
    for i in range(len(message)):
        if message[i] == chr(32):
            encrypted_text += " "
        else:
            counter = 0
            for j in range(len(alpha)):
                if message[i] == alpha[j]:
                    encrypted_text += alpha_key[counter]
                    break
                else:
                    counter += 1
    return encrypted_text


def decryption(word,alpha_key, alpha):
    
    message = word.upper()

    decrypted_text = ""
    for i in range(len(message)):
        if message[i] == chr(32):
            decrypted_text += " "
        else:
            counter = 0
            for j in range(len(alpha)):
                if message[i] == alpha_key[j]:
                    decrypted_text += alpha[counter]
                    break
                else:
                    counter += 1
    return decrypted_text


matrix = read_text('untitled.txt', collection = False)

def main():     
                
    alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    alpha_key = keyword_picker()

    choice = int(input("1. Encryption\n2. Decryption\nChoose(1,2): "))
    
    if choice == 1:
        words = read_text('dataset.txt')
        start_time = time.time()
        print("---Encrypting---")
        words=words.map(lambda x: encryption(x,alpha_key,alpha)).compute()
        end_time=time.time()
        print("Execution time= ",end_time-start_time)

    elif choice == 2:
        words=dd.read_table('dataset.txt')
        start_time = time.time()
        print("---Decrypting---")
        words=words.assign(enc=lambda x: decryptionpd(alpha_key,alpha,x))
        end_time=time.time()
        print("Execution time= ",end_time-start_time)
    else:
        print("Incorrect Choice")

if __name__ == "__main__":
    main()

Enter the Key
Enter key: keywords
1. Encryption
2. Decryption
Choose(1,2): 1
---Encrypting---
Execution time=  269.61434531211853
